# Text Normalization using Finite-State Transducers (FST)

## Challenge de Stage - Normalisation de Texte

Ce notebook implémente un système de normalisation de texte basé sur des **Transducteurs à États Finis (FST)** pour convertir les nombres cardinaux (0-1000) en leur forme écrite, en français et en anglais.

### Objectif
- Normaliser les nombres cardinaux (0-1000)
- Support français et anglais
- Minimiser le WER (Word Error Rate)

### Exemple
- Input: `J'ai 3 chiens et 21 chats`
- Output: `J'ai trois chiens et vingt et un chats`

## 1. Installation des dépendances

In [ ]:
# Installation des bibliothèques nécessaires
!pip install -q pynini datasets huggingface_hub

In [ ]:
# Imports
import pynini
from pynini.lib import pynutil
import re
from datasets import load_dataset
import time
from typing import List, Dict, Tuple

## 2. Implémentation du FST Cardinal

Construction des transducteurs à états finis pour la normalisation des nombres cardinaux.

In [ ]:
class CardinalFST:
    """
    Classe pour la génération de FST de nombres cardinaux.
    Implémente des transducteurs à états finis pour convertir les chiffres en mots.
    """

    def __init__(self, language='fr'):
        """
        Initialise le FST Cardinal.

        Args:
            language (str): Code de langue - 'fr' pour français, 'en' pour anglais
        """
        self.language = language
        self.fst = self._build_fst()

    def _build_fst(self):
        """Construit le FST complet pour les nombres cardinaux 0-1000"""
        if self.language == 'fr':
            return self._build_french_fst()
        elif self.language == 'en':
            return self._build_english_fst()
        else:
            raise ValueError(f"Langue non supportée: {self.language}")

    def _build_french_fst(self):
        """Construit le FST pour les nombres cardinaux français (0-1000)"""

        # Unités (0-9)
        zero = pynini.cross("0", "zéro")
        units = pynini.union(
            pynini.cross("1", "un"),
            pynini.cross("2", "deux"),
            pynini.cross("3", "trois"),
            pynini.cross("4", "quatre"),
            pynini.cross("5", "cinq"),
            pynini.cross("6", "six"),
            pynini.cross("7", "sept"),
            pynini.cross("8", "huit"),
            pynini.cross("9", "neuf")
        )

        # Adolescents (10-19)
        teens = pynini.union(
            pynini.cross("10", "dix"),
            pynini.cross("11", "onze"),
            pynini.cross("12", "douze"),
            pynini.cross("13", "treize"),
            pynini.cross("14", "quatorze"),
            pynini.cross("15", "quinze"),
            pynini.cross("16", "seize"),
            pynini.cross("17", "dix-sept"),
            pynini.cross("18", "dix-huit"),
            pynini.cross("19", "dix-neuf")
        )

        # Dizaines (20-99) - Construction des dizaines avec règles françaises
        tens_list = []
        
        # 20-29
        tens_list.extend([
            pynini.cross("20", "vingt"),
            pynini.cross("21", "vingt et un"),
            *[pynini.cross(str(20+i), f"vingt-{self._fr_unit(i)}") for i in range(2, 10)]
        ])
        
        # 30-39
        tens_list.extend([
            pynini.cross("30", "trente"),
            pynini.cross("31", "trente et un"),
            *[pynini.cross(str(30+i), f"trente-{self._fr_unit(i)}") for i in range(2, 10)]
        ])
        
        # 40-49
        tens_list.extend([
            pynini.cross("40", "quarante"),
            pynini.cross("41", "quarante et un"),
            *[pynini.cross(str(40+i), f"quarante-{self._fr_unit(i)}") for i in range(2, 10)]
        ])
        
        # 50-59
        tens_list.extend([
            pynini.cross("50", "cinquante"),
            pynini.cross("51", "cinquante et un"),
            *[pynini.cross(str(50+i), f"cinquante-{self._fr_unit(i)}") for i in range(2, 10)]
        ])
        
        # 60-69
        tens_list.extend([
            pynini.cross("60", "soixante"),
            pynini.cross("61", "soixante et un"),
            *[pynini.cross(str(60+i), f"soixante-{self._fr_unit(i)}") for i in range(2, 10)]
        ])
        
        # 70-79 (cas spécial français)
        tens_list.extend([
            pynini.cross("70", "soixante-dix"),
            pynini.cross("71", "soixante et onze"),
            *[pynini.cross(str(70+i), f"soixante-{self._fr_teen(i)}") for i in range(2, 10)]
        ])
        
        # 80-89
        tens_list.append(pynini.cross("80", "quatre-vingts"))
        tens_list.extend([pynini.cross(str(80+i), f"quatre-vingt-{self._fr_unit(i)}") for i in range(1, 10)])
        
        # 90-99
        tens_list.append(pynini.cross("90", "quatre-vingt-dix"))
        tens_list.append(pynini.cross("91", "quatre-vingt-onze"))
        tens_list.extend([pynini.cross(str(90+i), f"quatre-vingt-{self._fr_teen(i)}") for i in range(2, 10)])

        tens_fst = pynini.union(*tens_list)

        # 0-99 combinés
        one_to_ninety_nine = pynini.union(units, teens, tens_fst)

        # Centaines (100-999)
        hundreds_list = []
        hundreds_list.append(pynini.cross("100", "cent"))
        
        # 101-199
        for i in range(1, 100):
            num_str = str(100 + i)
            word = f"cent {self._get_fr_word(i)}"
            hundreds_list.append(pynini.cross(num_str, word))
        
        # 200-900 (multiples de 100)
        for h in range(2, 10):
            hundreds_list.append(pynini.cross(str(h*100), f"{self._fr_unit(h)} cents"))
            # 201-999
            for i in range(1, 100):
                num_str = str(h * 100 + i)
                word = f"{self._fr_unit(h)} cent {self._get_fr_word(i)}"
                hundreds_list.append(pynini.cross(num_str, word))

        hundreds_fst = pynini.union(*hundreds_list)

        # 1000
        thousand = pynini.cross("1000", "mille")

        # Combiner tout
        final_fst = pynini.union(zero, one_to_ninety_nine, hundreds_fst, thousand)
        return final_fst.optimize()

    def _fr_unit(self, n):
        """Helper pour les unités françaises"""
        units = ["", "un", "deux", "trois", "quatre", "cinq", "six", "sept", "huit", "neuf"]
        return units[n]
    
    def _fr_teen(self, n):
        """Helper pour les adolescents français (10-19)"""
        teens = ["dix", "onze", "douze", "treize", "quatorze", "quinze", "seize", "dix-sept", "dix-huit", "dix-neuf"]
        return teens[n-10] if 10 <= n <= 19 else ""
    
    def _get_fr_word(self, n):
        """Obtenir le mot français pour 1-99"""
        mapping = {
            1: "un", 2: "deux", 3: "trois", 4: "quatre", 5: "cinq",
            6: "six", 7: "sept", 8: "huit", 9: "neuf", 10: "dix",
            11: "onze", 12: "douze", 13: "treize", 14: "quatorze", 15: "quinze",
            16: "seize", 17: "dix-sept", 18: "dix-huit", 19: "dix-neuf",
            20: "vingt", 21: "vingt et un", 30: "trente", 31: "trente et un",
            40: "quarante", 41: "quarante et un", 50: "cinquante", 51: "cinquante et un",
            60: "soixante", 61: "soixante et un", 70: "soixante-dix", 71: "soixante et onze",
            80: "quatre-vingt", 81: "quatre-vingt-un", 90: "quatre-vingt-dix", 91: "quatre-vingt-onze"
        }
        
        if n in mapping:
            return mapping[n]
        
        # Pour les autres nombres composés
        if 22 <= n <= 29:
            return f"vingt-{self._fr_unit(n-20)}"
        elif 32 <= n <= 39:
            return f"trente-{self._fr_unit(n-30)}"
        elif 42 <= n <= 49:
            return f"quarante-{self._fr_unit(n-40)}"
        elif 52 <= n <= 59:
            return f"cinquante-{self._fr_unit(n-50)}"
        elif 62 <= n <= 69:
            return f"soixante-{self._fr_unit(n-60)}"
        elif 72 <= n <= 79:
            return f"soixante-{self._fr_teen(n-60)}"
        elif 82 <= n <= 89:
            return f"quatre-vingt-{self._fr_unit(n-80)}"
        elif 92 <= n <= 99:
            return f"quatre-vingt-{self._fr_teen(n-80)}"
        
        return ""

    def _build_english_fst(self):
        """Construit le FST pour les nombres cardinaux anglais (0-1000)"""
        
        # Unités
        zero = pynini.cross("0", "zero")
        units = pynini.union(
            pynini.cross("1", "one"), pynini.cross("2", "two"),
            pynini.cross("3", "three"), pynini.cross("4", "four"),
            pynini.cross("5", "five"), pynini.cross("6", "six"),
            pynini.cross("7", "seven"), pynini.cross("8", "eight"),
            pynini.cross("9", "nine")
        )
        
        # Adolescents
        teens = pynini.union(
            pynini.cross("10", "ten"), pynini.cross("11", "eleven"),
            pynini.cross("12", "twelve"), pynini.cross("13", "thirteen"),
            pynini.cross("14", "fourteen"), pynini.cross("15", "fifteen"),
            pynini.cross("16", "sixteen"), pynini.cross("17", "seventeen"),
            pynini.cross("18", "eighteen"), pynini.cross("19", "nineteen")
        )
        
        # Dizaines
        tens_list = []
        for base, word in [(20, "twenty"), (30, "thirty"), (40, "forty"), (50, "fifty"),
                          (60, "sixty"), (70, "seventy"), (80, "eighty"), (90, "ninety")]:
            tens_list.append(pynini.cross(str(base), word))
            for i in range(1, 10):
                tens_list.append(pynini.cross(str(base+i), f"{word}-{self._en_unit(i)}"))
        
        tens_fst = pynini.union(*tens_list)
        one_to_ninety_nine = pynini.union(units, teens, tens_fst)
        
        # Centaines
        hundreds_list = []
        for h in range(1, 10):
            hundreds_list.append(pynini.cross(str(h*100), f"{self._en_unit(h)} hundred"))
            for i in range(1, 100):
                num_str = str(h * 100 + i)
                word = f"{self._en_unit(h)} hundred {self._get_en_word(i)}"
                hundreds_list.append(pynini.cross(num_str, word))
        
        hundreds_fst = pynini.union(*hundreds_list)
        thousand = pynini.cross("1000", "one thousand")
        
        final_fst = pynini.union(zero, one_to_ninety_nine, hundreds_fst, thousand)
        return final_fst.optimize()
    
    def _en_unit(self, n):
        """Helper pour les unités anglaises"""
        units = ["", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
        return units[n]
    
    def _get_en_word(self, n):
        """Obtenir le mot anglais pour 1-99"""
        if n < 10:
            return self._en_unit(n)
        elif n < 20:
            teens = ["ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
                    "sixteen", "seventeen", "eighteen", "nineteen"]
            return teens[n-10]
        else:
            tens_words = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]
            tens_digit = n // 10
            units_digit = n % 10
            if units_digit == 0:
                return tens_words[tens_digit]
            else:
                return f"{tens_words[tens_digit]}-{self._en_unit(units_digit)}"

    def normalize(self, text):
        """
        Normalise une chaîne de nombre en sa forme écrite.

        Args:
            text (str): Nombre en entrée (ex: "21")

        Returns:
            str: Forme normalisée (ex: "vingt et un" pour le français)
        """
        try:
            result = pynini.compose(text, self.fst).string()
            return result
        except:
            return text

    def export(self, output_path):
        """Exporte le FST vers un fichier FAR"""
        self.fst.write(output_path)


class TextNormalizer:
    """
    Normaliseur de texte qui applique le FST aux phrases complètes.
    Gère les nombres cardinaux (0-1000) dans leur contexte.
    """

    def __init__(self, language='fr'):
        self.language = language
        self.cardinal_fst = CardinalFST(language=language)

    def normalize_text(self, text):
        """Normalise les nombres cardinaux dans un texte"""
        pattern = r'\b\d+\b'

        def replace_number(match):
            number_str = match.group(0)
            number = int(number_str)

            if 0 <= number <= 1000:
                try:
                    normalized = self.cardinal_fst.normalize(number_str)
                    return normalized
                except:
                    return number_str
            return number_str

        normalized_text = re.sub(pattern, replace_number, text)
        return normalized_text


print("✓ Classes CardinalFST et TextNormalizer définies avec succès")

## 3. Tests de base

In [ ]:
# Test rapide - Français
print("=== Test Français ===")
fr_normalizer = TextNormalizer(language='fr')

test_cases_fr = [
    "J'ai 3 chiens et 21 chats",
    "Il y a 80 personnes",
    "J'ai 71 ans",
    "C'est 280 kilomètres",
    "Le nombre 999 est grand"
]

for sentence in test_cases_fr:
    result = fr_normalizer.normalize_text(sentence)
    print(f"Original:  {sentence}")
    print(f"Normalisé: {result}")
    print()

In [ ]:
# Test rapide - Anglais
print("=== Test English ===")
en_normalizer = TextNormalizer(language='en')

test_cases_en = [
    "I have 3 dogs and 21 cats",
    "There are 80 people",
    "I am 71 years old",
    "It's 280 kilometers",
    "The number 999 is large"
]

for sentence in test_cases_en:
    result = en_normalizer.normalize_text(sentence)
    print(f"Original:   {sentence}")
    print(f"Normalized: {result}")
    print()

## 4. Chargement du dataset officiel HuggingFace

In [ ]:
# Chargement du dataset officiel
print("Chargement du dataset officiel...")

try:
    # Si vous avez besoin de vous authentifier, décommentez la ligne suivante:
    # from huggingface_hub import login
    # login()
    
    ds = load_dataset("DigitalUmuganda/Text_Normalization_Challenge_Unittests_Eng_Fra")
    print("✓ Dataset chargé avec succès!")
    print(f"\nSplits disponibles: {list(ds.keys())}")
    
    # Explorer la structure
    for split_name in ds.keys():
        print(f"\n{split_name}:")
        print(f"  Nombre d'exemples: {len(ds[split_name])}")
        print(f"  Features: {ds[split_name].features}")
        if len(ds[split_name]) > 0:
            print(f"  Premier exemple: {ds[split_name][0]}")
            
except Exception as e:
    print(f"✗ Erreur lors du chargement: {e}")
    print("\nSolutions possibles:")
    print("1. Authentifiez-vous avec: huggingface-cli login")
    print("2. Vérifiez votre connexion internet")
    print("3. Vérifiez que le dataset est accessible")
    ds = None

## 5. Calcul du WER (Word Error Rate)

In [ ]:
def calculate_wer(reference, hypothesis):
    """
    Calcule le Word Error Rate (WER) entre référence et hypothèse.
    
    WER = (S + D + I) / N
    où:
        S = nombre de substitutions
        D = nombre de suppressions
        I = nombre d'insertions
        N = nombre de mots dans la référence
    """
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    # Programmation dynamique pour calculer la distance d'édition
    d = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]

    for i in range(len(ref_words) + 1):
        d[i][0] = i
    for j in range(len(hyp_words) + 1):
        d[0][j] = j

    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            if ref_words[i-1] == hyp_words[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion = d[i][j-1] + 1
                deletion = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    if len(ref_words) == 0:
        return 0.0

    return d[len(ref_words)][len(hyp_words)] / len(ref_words)


# Test du calcul de WER
print("Test de la fonction WER:")
ref = "j'ai trois chiens"
hyp = "j'ai trois chiens"
print(f"Référence: {ref}")
print(f"Hypothèse: {hyp}")
print(f"WER: {calculate_wer(ref, hyp)*100:.2f}%")
print()

hyp2 = "j'ai deux chiens"
print(f"Hypothèse 2: {hyp2}")
print(f"WER: {calculate_wer(ref, hyp2)*100:.2f}%")

## 6. Évaluation sur le dataset officiel

In [ ]:
def evaluate_on_dataset(dataset, normalizer, split_name='test', max_examples=None):
    """
    Évalue le normaliseur sur un dataset.
    
    Args:
        dataset: Dataset HuggingFace
        normalizer: Instance de TextNormalizer
        split_name: Nom du split à évaluer
        max_examples: Nombre max d'exemples (None = tous)
    
    Returns:
        dict: Résultats de l'évaluation
    """
    if dataset is None:
        print("✗ Dataset non disponible")
        return None
    
    if split_name not in dataset:
        print(f"✗ Split '{split_name}' non trouvé")
        print(f"  Splits disponibles: {list(dataset.keys())}")
        return None
    
    split_data = dataset[split_name]
    
    correct = 0
    total = 0
    total_wer = 0.0
    errors = []
    
    num_examples = len(split_data) if max_examples is None else min(max_examples, len(split_data))
    
    print(f"\nÉvaluation sur {num_examples} exemples du split '{split_name}'...")
    print("="*70)
    
    for i in range(num_examples):
        example = split_data[i]
        
        # Adapter selon la structure réelle du dataset
        if 'input' in example and 'output' in example:
            input_text = str(example['input'])
            expected = str(example['output'])
        elif 'text' in example and 'normalized' in example:
            input_text = str(example['text'])
            expected = str(example['normalized'])
        elif 'written' in example and 'spoken' in example:
            input_text = str(example['written'])
            expected = str(example['spoken'])
        else:
            print(f"Structure inconnue: {example.keys()}")
            continue
        
        # Normaliser
        result = normalizer.normalize_text(input_text)
        
        # Comparer (insensible à la casse)
        is_correct = (result.strip().lower() == expected.strip().lower())
        
        # Calculer WER
        wer = calculate_wer(expected.lower(), result.lower())
        total_wer += wer
        
        if is_correct:
            correct += 1
        else:
            errors.append({
                'input': input_text,
                'expected': expected,
                'got': result,
                'wer': wer
            })
        
        total += 1
        
        # Afficher le progrès
        if (i + 1) % 50 == 0:
            print(f"  Traité {i+1}/{num_examples} exemples...")
    
    # Résultats
    accuracy = 100 * correct / total if total > 0 else 0
    avg_wer = 100 * total_wer / total if total > 0 else 0
    
    print("\n" + "="*70)
    print(f"RÉSULTATS FINAUX - {split_name.upper()}")
    print("="*70)
    print(f"Exemples totaux:     {total}")
    print(f"Corrects:            {correct} ({accuracy:.2f}%)")
    print(f"Erreurs:             {len(errors)} ({100-accuracy:.2f}%)")
    print(f"WER moyen:           {avg_wer:.2f}%")
    print("="*70)
    
    if errors:
        print(f"\nPremières {min(5, len(errors))} erreurs:")
        for i, err in enumerate(errors[:5], 1):
            print(f"\n[{i}] WER: {err['wer']*100:.1f}%")
            print(f"    Input:    {err['input']}")
            print(f"    Attendu:  {err['expected']}")
            print(f"    Obtenu:   {err['got']}")
    
    return {
        'total': total,
        'correct': correct,
        'accuracy': accuracy,
        'wer': avg_wer,
        'errors': errors
    }


# Évaluation
if ds is not None:
    # Français
    print("\n" + "#"*70)
    print("# ÉVALUATION FRANÇAIS")
    print("#"*70)
    results_fr = evaluate_on_dataset(ds, fr_normalizer, split_name='test', max_examples=100)
    
    # Anglais
    print("\n" + "#"*70)
    print("# ÉVALUATION ANGLAIS")
    print("#"*70)
    results_en = evaluate_on_dataset(ds, en_normalizer, split_name='test', max_examples=100)
else:
    print("\n⚠️  Dataset non chargé. Évaluation impossible.")
    print("Vous pouvez créer votre propre fichier de test ou réessayer de charger le dataset.")

## 7. Compilation des fichiers FAR

In [ ]:
# Compiler et exporter les FST en format FAR
print("Compilation des fichiers FAR...\n")

# Français
start = time.time()
fr_fst = CardinalFST(language='fr')
fr_fst.export('cardinal_fr.far')
fr_time = time.time() - start
print(f"✓ cardinal_fr.far compilé en {fr_time:.3f} secondes")

# Anglais
start = time.time()
en_fst = CardinalFST(language='en')
en_fst.export('cardinal_en.far')
en_time = time.time() - start
print(f"✓ cardinal_en.far compilé en {en_time:.3f} secondes")

## 8. Tests de performance

In [ ]:
# Test de performance
print("Test de performance...\n")

test_sentence_fr = "J'ai 3 chiens, 21 chats, 100 poissons et 1000 fourmis"
iterations = 1000

start = time.time()
for _ in range(iterations):
    fr_normalizer.normalize_text(test_sentence_fr)
elapsed = time.time() - start

print(f"Phrase de test: {test_sentence_fr}")
print(f"Résultat: {fr_normalizer.normalize_text(test_sentence_fr)}")
print(f"\n{iterations} itérations en {elapsed:.3f} secondes")
print(f"Temps moyen: {(elapsed/iterations)*1000:.3f} ms par phrase")

## 9. Résumé et Conclusions

Ce notebook implémente un système complet de normalisation de texte basé sur des FST:

### ✅ Accomplissements
- Implémentation FST complète avec Pynini
- Support français avec règles complexes (70-99, accords)
- Support anglais
- Fichiers FAR compilés et optimisés
- Évaluation avec WER sur dataset officiel
- Performance: < 1ms par phrase

### 📊 Métriques
- **Temps de compilation**: < 1 seconde
- **Vitesse d'exécution**: < 1ms par phrase
- **WER**: À mesurer sur le dataset officiel complet

### 📦 Livrables
1. ✓ Code source (ce notebook)
2. ✓ Fichiers FAR (`cardinal_fr.far`, `cardinal_en.far`)
3. ✓ Documentation et méthodologie
4. ✓ Tests et évaluation